In [1]:
import warnings
warnings.filterwarnings("ignore")

### Dec Crack Spreads from 2011 to 2019

In [2]:

import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr

# importlib.reload(build_db)

### important global variables

In [3]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'


In [4]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,width=800,height=400,
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None,
                modebar_orientation='v',modebar_color='grey',
                legend_x=None,legend_y=None,
                title_y_pos = 0.9,
                title_x_pos = 0.5):
    
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout

    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        autosize=True,
#         autosize=False,
#         width=width,
#         height=height,
        margin=Margin(
            b=100
        ),
        modebar={'orientation': modebar_orientation,'bgcolor':modebar_color}
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':title_y_pos,
            'x':title_x_pos,
            'xanchor': 'center',
            'yanchor': 'top'})
    if (legend_x is not None) and (legend_y is not None):
        fig.update_layout(legend=dict(x=legend_x, y=legend_y))
    return fig


def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

#### get all contracts in the options database

In [5]:
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


  sec_db
futtab max date:         max
0  20211214
opttab max date:         max
0  20211214


In [6]:
all_commods_sql = f"select distinct substring(symbol,1,2) commod from {futtab}"
df_all_commods = pga.get_sql(all_commods_sql)

In [7]:
print(df_all_commods.commod.values)

['EW' 'VI' 'SQ' 'BG' 'YI' 'LC' 'US' 'ZQ' 'SB' 'EC' 'D6' 'SL' 'EY' 'J6'
 'SY' 'SK' 'ZM' 'MJ' 'YC' 'GH' 'ZB' 'SX' 'YM' 'SN' 'DX' 'KE' 'PA' 'CN'
 'DY' 'KW' 'PB' 'CF' 'CL' 'IR' 'OJ' 'LV' 'M6' 'FM' 'II' 'GJ' 'EF' 'ZS'
 'QL' 'LH' 'SI' 'PC' 'YU' 'DS' 'LZ' 'DA' 'IM' 'JZ' 'LQ' 'BP' 'GU' 'NX'
 'SZ' 'CK' 'L6' 'A6' 'WI' 'RR' 'MD' 'RS' 'SM' 'OU' 'N6' 'RA' 'NK' 'AE'
 'RJ' 'DB' 'HR' 'ON' 'MX' 'FC' 'YK' 'ES' 'LG' 'RB' 'NY' 'ZW' 'ZC' 'LU'
 'TP' 'NE' 'E6' 'FZ' 'EJ' 'OH' 'XG' 'CD' 'SR' 'FS' 'NI' 'SF' 'GB' 'PD'
 'GC' 'JM' 'XM' 'NV' 'GE' 'CH' 'LN' 'HE' 'SV' 'YV' 'CA' 'WK' 'NF' 'VS'
 'WH' 'GZ' 'ZF' 'GG' 'CZ' 'ZD' 'TU' 'WN' 'UL' 'MW' 'NG' 'KS' 'FV' 'YW'
 'AD' 'RM' 'HF' 'B6' 'MQ' 'T6' 'LE' 'LB' 'FU' 'AP' 'WU' 'LM' 'SH' 'OK'
 'DL' 'YG' 'YT' 'FH' 'UD' 'ZG' 'KC' 'GI' 'WT' 'CI' 'LJ' 'TI' 'LF' 'ZL'
 'OZ' 'BR' 'S6' 'PL' 'CT' 'CC' 'HG' 'SU' 'HO' 'NJ' 'ZR' 'CB' 'JX' 'FX'
 'SD' 'QR' 'ED' 'CG' 'WZ' 'ND' 'SP' 'TY' 'KL' 'HS' 'XU' 'ZT' 'BO' 'ZK'
 'SW' 'LK' 'ZI' 'NQ' 'JY' 'CE' 'LX' 'EM' 'LS' 'GF' 'SA' 'BT' 'XH' 'GM'
 'ZN' 

In [8]:
commod = 'CL'
price_sql = f"select symbol, settle_date, close from {futtab} f where substring(f.symbol,1,2)='{commod}'"
df_cl = pga.get_sql(price_sql)

commod = 'HO'
price_sql = f"select symbol, settle_date, close from {futtab} f where substring(f.symbol,1,2)='{commod}'"
df_ho = pga.get_sql(price_sql)
df_cl = df_cl.rename(columns={'close':'cl'})
df_cl['myy'] = df_cl.symbol.str.slice(2,)
df_ho = df_ho.rename(columns={'close':'ho'})
df_ho['myy'] = df_ho.symbol.str.slice(2,)
df_crack = df_cl[['myy','settle_date','cl']].merge(
    df_ho[['myy','settle_date','ho']],
    on=['myy','settle_date'],
    how='inner'
)

In [9]:
df_crack['crack']= df_crack.ho*42 - df_crack.cl
df_crack

2022-02-03 16:52:11,210 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.


,myy,settle_date,cl,ho,crack
0,Z99,20100901,73.91,2.0248,11.1316
1,V10,20100901,73.91,2.0411,11.8162
2,X10,20100901,75.36,2.0632,11.2944
3,Z10,20100901,76.86,2.0871,10.7982
4,F11,20100901,77.90,2.1139,10.8838
...,...,...,...,...,...
110612,U24,20210730,57.29,2.0057,26.9494
110613,V24,20210730,57.05,2.0090,27.3280
110614,X24,20210730,56.83,2.0066,27.4472
110615,Z24,20210730,56.60,1.9994,27.3748


In [13]:
df_crack2[df_crack2.myy=='Z12'].tail()

,myy,settle_date,cl,ho,crack
14313,Z12,20121112,85.57,2.9992,40.3964
14352,Z12,20121113,85.38,2.9608,38.9736
14391,Z12,20121114,86.32,2.9882,39.1844
14430,Z12,20121115,85.45,2.9735,39.4370
14469,Z12,20121116,86.67,2.9868,38.7756


In [14]:
commod_month = 'H'

In [15]:
df_crack2 = df_crack[df_crack.myy.str.contains(f'{commod_month}[1][0-9]')]
def print_crack(df):
    return plotly_plot(df[['settle_date','crack']],x_column='settle_date',
                      plot_title=f'crack: {str(df.myy.iloc[0])}')
    
# figs = df_crack2.groupby('myy').apply(print_crack)

groups = df_crack2.groupby('myy')
d = [(name,len(df.iloc[-200:].crack.values),df.iloc[-200:].crack.values) for name,df in groups if len(df)>=200]
d2= {name:df.iloc[-200:].crack.values for name,df in groups if len(df)>=200}
df_crack3 = pd.DataFrame(d2)
df_crack3['days'] = df_crack3.index.values
iplot(
    plotly_plot(
        df_crack3,x_column='days',
        plot_title=f"Dec Heat Crack Spreads from {commod_month}11 to {commod_month}19",
        y_left_label='Crack Price $/Barrel'
    )
)
    



## END